## Начало из прошлой лекции

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [2]:
with open("../female-names.txt", "r") as f_in:
    female_names = f_in.readlines()

with open("../male-names.txt", "r") as f_in:
    male_names = f_in.readlines()

female_names = [x.lower().strip() for x in female_names]
male_names = [x.lower().strip() for x in male_names]
male_names.extend(female_names)

In [4]:
names = [x + "." for x in male_names]

In [5]:
vocab = sorted(list(set("".join(names))), key=lambda v: "\t" if v == "." else v)
vocab_size = len(vocab)

In [6]:
char_to_index = {char: index for index, char in enumerate(vocab)}
index_to_char = {index: char for char, index in char_to_index.items()}

def tokenize(char):
    return char_to_index.get(char, 0) 

def untokenize(index):
    return index_to_char.get(index, ".")